# Radiation coupled with convection

![Figure 1](./figures/07_radiation_convect_prism.png)

> Figures 1. a) Triagular prism. b) Heat flow rate balance on a surface.

## Problem

Let's consider a triangular prism of infinite lenght (Figure 1a) with the characteristics:
- surface 1: transparnt in short-wave, $\tau_{SW}=1$, opaque $\tau_{LW} = 0$ and absorbant $\alpha_{LW} = 0.8$ in long-wave.
- surfaces 2 and 3: opaque $\tau_{SW}=1$ and absorbant $\alpha_{SW} = 0.8$ in short-wave and opaque $\tau_{LW} = 0$ and absorbant $\alpha_{LW} = 0.8$ in long-wave.

The short wave (solar) radiation on the walls 1 and 2 are $E^o_1 = 120 \, \mathrm{W/m²}$ and $E^o_2 = 100 \, \mathrm{W/m²}$, respectively.

The indoor temperature of the prism is controlled at $T_i = 20\,\mathrm{°C}$. The heat convection coefficient between the walls and the indoor air is $h_i = 5 \, \mathrm{W/m²K}$ for all wals.

The temperature of each wall is considered homogeneous. The heat balance of a wall is represented in Figure 1b.
The heat flow rate by conduction is negligible for all walls, $\varphi_{cd} = 0$ (Figure 1b).

In this conditions, find the temperatures of the walls.

In [1]:
import numpy as np
 
np.set_printoptions(precision=3)

# Stefan-Bolzmann constant
σ = 5.670e-8    # W / (m2 K4)

# Transmission coefficients - long-wave (LW)
τ = np.zeros(3)
τ[0] = 1
τ = np.diag(τ)

# Coefficients d'absorbtion CLO
α = np.zeros(3)
α[1] = α[2] = 0.8
α = np.diag(α)

# Coefficient de reflexion en CLO
ρ = 1 - (α + τ)

Ta = 20     # °C, température air intérieure
Ta = Ta * np.ones(3)
h = 5       # W/(m2 K)
h = h * np.eye(3)

# Identity matrix
I = np.eye(3)

## Solution

### View factors
$F_{1,1} = F_{2,2} = F_{3,3} $ - the surfaces are flat.

$F_{1,2} = F_{3,1}$ - due to symetry.

From the complementarity relation of view factors, $\sum_{j} F_{1, j} = F_{1, 1} + F_{1, 2} + F_{1, 3} = 1$, is results that $F_{1, 2} = F_{1, 3} = 1/2$.

From the reciprocity relation, $F_{1,2}S_1 = F_{2,1}S_2$ and from the relation between surface areas, $S_1 = \sqrt{2}S_2$, it results that $ \sqrt{2} F_{1, 2} = F_{2, 1}$, i.e. $F_{2, 1} = \sqrt{2} / 2$.

From the complementarity relation of view factors, $\sum_{j} F_{2, j} = F_{2, 1} + F_{2, 2} + F_{2, 3} = 1$, is results that $F_{2, 3} = 1 - F_{2, 1} = 1 - \sqrt{2} / 2$.

Due to symetry, $F_{3, 2} = F_{2, 3}$ 

In [2]:
F = np.zeros([3, 3])
F[0, 1] = F[0, 2] = 1 / 2
F[1, 0] = F[2, 0] = np.sqrt(2) / 2
F[1, 2] = F[2, 1] = 1 - F[1, 0]
print("F = ")
print(F)

F = 
[[0.    0.5   0.5  ]
 [0.707 0.    0.293]
 [0.707 0.293 0.   ]]


### Short-wave (solar) radiation

The set of previous equation for short-wave solar radiation:

$$
\begin{bmatrix}
1 - \rho_1 F_{1,1} & - \rho_2 F_{1,2}    & - \rho_n F_{1,3}\\ 
- \rho_1 F_{2,1}   & 1 - \rho_2 F_{2,2}  & - \rho_n F_{2,n3} \\ 
- \rho_1 F_{n,1}   & - \rho_2 F_{n,1}    & 1 - \rho_n F_{n,n}
\end{bmatrix} \begin{bmatrix}
E_1\\ 
E_2\\ 
E_3
\end{bmatrix} = \begin{bmatrix}
E_{1}^{o}\\ 
E_{2}^{o}\\  
E_{3}^{o}
\end{bmatrix}
$$

or

$$(I - \rho \circ F) E = E^o$$

The unknown total [irradiances](https://en.m.wikipedia.org/wiki/Irradiance) on walls, in W/m², are then

$$ E = (I - \rho \circ F)^{-1} E^o$$

where:

- the symbol $\circ$ represents the [Hadamard (or element-wise) product](https://en.wikipedia.org/wiki/Hadamard_product_(matrices));

$I =\begin{bmatrix}
1 & 0 & 0 \\ 
0 & 1 & 0 \\ 
0 & 0 & 1 
\end{bmatrix}, $ is the [identity matrix](https://en.m.wikipedia.org/wiki/Identity_matrix);

$\rho = \begin{bmatrix}
\rho_1\\ 
\rho_2\\ 
\rho_3
\end{bmatrix}$ - vector of [reflectances](https://en.m.wikipedia.org/wiki/Reflectance), $0 \le \rho_{i,j} \le 1$;

$F = \begin{bmatrix}
F_{1,1} & F_{1,2} & F_{1,3}\\ 
F_{2,1} & F_{2,2} & F_{2,3} \\ 
F_{3,1} & F_{3,2} & F_{3,3}
\end{bmatrix}$ - matrix of [view factors](https://en.m.wikipedia.org/wiki/View_factor), $0 \le F_{i,j} \le 1$;

$E^o = \begin{bmatrix}
E_{1}^{o}\\ 
E_{2}^{o}\\ 
E_{3}^{o}
\end{bmatrix}$ - vector of [direct solar irradiances](https://en.m.wikipedia.org/wiki/Solar_irradiance), W/m²;

$E = \begin{bmatrix}
E_1\\ 
E_2\\  
E_3
\end{bmatrix}$ - vector of unknown total irradiances, W/m².


In [3]:
E0 = np.zeros(3)
E0[1], E0[2] = 120, 100     # W/m2

E = np.linalg.inv((I - np.diag(ρ) * F)) @ E0
print("E = ")
print(E, 'W/m²')

E = 
[ 23.369 126.291 107.398] W/m²


Short-wave heat flow rate absorbed by the surfaces:
$$\varphi = \alpha E$$

In [4]:
φSW = np.diag(α) * E
print('φ_SW = ')
print(φSW, 'W/m²')

φ_SW = 
[  0.    101.033  85.918] W/m²


### Long-wave (infrared) thermal radiation

Long-wave absorbtion coefficinets of the surfaces are $\alpha_{LW,0} = \alpha_{LW,1} = \alpha_{LW,2} = 0.8$.

In [5]:
α = 0.8 * np.ones(3)
α = np.diag(α)
ε = α       # Kirchhoff law

The net flow rate density emmitted by a surface is:
$$\varphi_{LW, i} = \varepsilon_i M^o_i - \alpha_i E_{LW,i}$$
where:
- $M^o_i$ - [emmitance](https://en.m.wikipedia.org/wiki/Radiant_exitance) of the surface, $M^o_i = \sigma T_i^4$;
- $E_{LW,i}$ - global long wave radiation (luminace) received by the surface;
- $\alpha_i$ - coefficent of absortivity in long wave;
- $\varepsilon_i$ - coeffient of emmissivity in long wave.

The global long wave radiation (luminace) received by the surface is the sum of the emmitted and reflected flow, refe

$$S_i E_{LW,i} = \sum_j F_{j, i} S_j (\varepsilon_jM^o_j + \rho_j E_{LW, j}) = \sum_j F_{j, i} S_j J_j$$

where:
- $J_j$ is the [radiosity](https://en.m.wikipedia.org/wiki/Radiosity_(radiometry)).

Taking int account the relation of [reciprocity](https://en.wikipedia.org/wiki/View_factor#Reciprocity), $F_{i, j} S_i = F_{j, i} S_j$, it results that

$$E_{LW,i} = \sum_j F_{i, j} (\varepsilon_jM^o_j + \rho_j E_{LW, j})$$

For opaque walls, $\tau_i = 0$. Then $\rho_i = 1 - \alpha_i$ and, since $\alpha_i = \varepsilon_i$, $\rho_i = 1 - \varepsilon_i$. Therefore,

$$E_{LW,i} - \sum_j F_{i, j} (1 - \varepsilon_j) E_{LW,j} = \sum_j F_{i,j} \varepsilon_j M^o_j$$

In matrix form, this equation is:

$$ (I - F(I - \varepsilon))E_{LW} = F \varepsilon M^o$$

In our case, the matrix $(I - F(I - \varepsilon))$ is:

In [6]:
print(I - F @ (I - ε))

[[ 1.    -0.1   -0.1  ]
 [-0.141  1.    -0.059]
 [-0.141 -0.059  1.   ]]


and the matrix $F \varepsilon$ is:

In [7]:
print(F @ ε)

[[0.    0.4   0.4  ]
 [0.566 0.    0.234]
 [0.566 0.234 0.   ]]


From this equation, it results the vector of long wave radiation on surfaces:

$$E_{LW} = [(I - F(I - \varepsilon))]^{-1} F \varepsilon M^o$$

By substituting $E_{LW}$ into

$$\varphi_{LW, i} = \varepsilon_i M^o_i - \alpha_i E_{LW,i}$$

we obtain

$$\varphi_{LW} =  \varepsilon \{I - [I - F (I - \varepsilon)]^{-1}\ F \varepsilon\}  M^o = K M^o$$

In our case, the matrix $K = \varepsilon \{I - [I - F (I - \varepsilon)]^{-1}\ F \varepsilon\}$ is: 

In [8]:
K = ε @ (I - np.linalg.inv(I - F @ (I - ε)) @ F @ ε)
print("K = ")
print(K)

K = 
[[ 0.701 -0.35  -0.35 ]
 [-0.496  0.736 -0.241]
 [-0.496 -0.241  0.736]]


The equation of $\varphi_{LW} = K M^o$ is linear in emmitances $M^o$ but not linear in temperatures, since $M^o_i = \sigma T_i^4$. In order to linearize the equation $\varphi_{LW} = K \sigma T^4$, where 

$$T^4 = \begin{bmatrix}
T_0^4\\ 
T_1^4\\ 
T_2^4
\end{bmatrix}$$

is the vector of (absolote values) temperatures of the surfaces.

Let's consider the particular case in which all temperatures have the same value $\bar{T}$. In this case, the heat flow rate between the surfaces is zero,

$$\bar \varphi_{LW} = K \sigma \bar T^4 = 0$$

Therefore,

$$\varphi_{LW} - \bar \varphi_{LW} = \varphi_{LW} = K \sigma (T^4 - \bar T^4)$$

Taking into account that

$$T_i^4 - \bar T_i^4 = (T_i^2 + \bar T_i^2 )(T_i + \bar T_i )(T_i - \bar T_i )$$

The linearized equation for long-wave radiation is then:

$$\varphi_{LW} = K M^o = \sigma K H (T - \bar T) = \sigma K H \theta$$

where the matris $H$ is diagonal with elements
$$H_i = (T_i^2 + \bar T_i^2 )(T_i + \bar T_i)$$

Solving this equation reausires an initial guess of $\bar T_i$. If we consider that $T_i  \approx \bar T_i$ and that $\bar T_i = \bar T = 273.15 \, \mathrm K$ for all $i$, then

$$T_i^4 - \bar T_i^4 \approx 4 \bar T^3$$

and

$$\varphi_{LW} = K M^o \approx 4 \sigma \bar T^3 K \theta$$

where 

$$\theta = \begin{bmatrix}
\theta_0\\ 
\theta_1\\ 
\theta_2
\end{bmatrix}$$

are the surface temperatures expressed in Celsius.

For our example, the diagonal matrix $H \approx 4 \sigma \bar T^3 I$ is:

In [9]:
T0 = 273.15        # K, temperature
H = 4 * σ * T0**3 * np.eye(3)
print(H)

[[4.622 0.    0.   ]
 [0.    4.622 0.   ]
 [0.    0.    4.622]]


The matrix $\sigma \bar T^3 K$ of the lierized expression of heat flow rates $$\varphi_{LW} = 4 \sigma \bar T^3 K \theta = L \theta$$ is:

In [10]:
L = H @ K
print("L = ")
print(L)

L = 
[[ 3.24  -1.62  -1.62 ]
 [-2.291  3.403 -1.113]
 [-2.291 -1.113  3.403]]


**Note:** For values of the temperature that are usual in buildings, i.e., $0°C \leq \bar T \leq 40 °C$, the values of $4 \sigma \bar T^3$ vary between 4.5 and 7.

In [12]:
T0 = 273.15 + np.array([0, 40])
print(4 * σ * T0**3)

[4.622 6.965]


### Coupled short-wave radiation, long-wave radiation and convection

From the heat flow balance for a surface $i$ shown in Figure 1b, it results that

$$\varphi_{cv} + \varphi_{SW} - \varphi_{LW} + \varphi_{cd} = 0 $$

where:
- $\varphi_{cv} = h_i (T_a - \theta)$ is the flow rate by convection;
- $\varphi_{SW}$ - short-wave radiation absorbed by the surfaces;
- $\varphi_{LW} = L \theta$ - long-wave radiation;
- $\varphi_{cd} = 0$ - flow-rate through conduction is zero in this case.

By substitution, it results:

$$h_i(T_a - \theta) + \varphi_{SW} - L \theta = 0 $$

which, by considering that the vector of temperatures is the uknown, becomes:

$$(L + hI) \theta = (\varphi_{SW} + h T_a)$$

By solving for surface temperatures $\theta$, it results:

$$\theta = (L + hI)^{-1} (\varphi_{SW} + h T_a)$$

In [11]:
θ = np.linalg.inv(L + h * I) @ (φSW + np.diag(h) * Ta)
print("θ = ")
print(θ, "°C")

θ = 
[25.751 35.422 33.834] °C
